In [ ]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')


In [ ]:
import os

# List everything in /kaggle/input
os.listdir('/kaggle/input')


In [ ]:
import os

dataset_path = '/kaggle/input/chest-exray-pneumonia-detection'
os.listdir(dataset_path)


In [ ]:
import os

dataset_path = '/kaggle/input/chest-exray-pneumonia-detection/chest_xray'
os.listdir(dataset_path)


MAKING IMPORTS

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
%matplotlib inline
!pip install seaborn
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img

**pretrained convolutionary neural network**

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions



model = Xception(
    weights="imagenet",
    input_shape=(299, 299, 3)
  )

TRANSFER LEARNING AND READING THE DATASET

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_ds = train_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/val',
    target_size=(150,150),
    batch_size=32,
    class_mode='binary'
)



In [ ]:
train_ds.class_indices


DATA VISUALIZATION

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
class_labels_numeric = train_ds.classes
class_names = list(train_ds.class_indices.keys())
descriptive_labels = [class_names[label] for label in class_labels_numeric]

sns.set_style('darkgrid')
plt.figure(figsize=(8, 6))
sns.countplot(x=descriptive_labels)
plt.title('Distribution of Classes in Training Dataset')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

NODEL TRAINING

In [ ]:
base_model = Xception(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)

base_model.trainable = False

CREATING A NEW TOP

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

In [ ]:
preds = model.predict(val_ds)


In [ ]:
preds.shape

In [ ]:
preds

In [ ]:
preds =model.predict(train_ds)

In [ ]:
preds.shape

In [ ]:
preds

COMPILING THE MODEL

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:

labels = {v: k for k, v in train_ds.class_indices.items()}


images, batch_labels = next(train_ds)


plt.figure(figsize = (5,5))
plt.imshow(images[0], cmap='gray')
plt.title(labels[batch_labels[0].astype(int)])


plt.figure(figsize = (5,5))
plt.imshow(images[-1], cmap='gray')
plt.title(labels[batch_labels[-1].astype(int)])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
history.history['accuracy']
#history.history['val_accuracy']


In [ ]:
#plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

ADJUST THE LEARNING RATE

In [ ]:
def make_model(learning_rate=0.01):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False



    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    outputs = keras.layers.Dense(1, activation='sigmoid')(vectors)
    model = keras.Model(inputs, outputs)



    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    loss = keras.losses.BinaryCrossentropy(from_logits=False)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [ ]:

scores = {}

for lr in [ 0.001, 0.01]:
    print(lr)

    model = make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history

    print()
    print()

In [ ]:
for lr, hist in scores.items():

    plt.plot(hist['val_accuracy'], label=('val=%s' % lr))

plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001

**CHECKPOINTING AND TRAINNING THE MODEL WITH CALLBACKS**

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001

model = make_model(learning_rate=learning_rate)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

LETS ADD ONE INNER DENSE LAYER

In [ ]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False


    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)

    outputs = keras.layers.Dense(1, activation='sigmoid')(inner)

    model = keras.Model(inputs, outputs)


    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.BinaryCrossentropy(from_logits=False)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [ ]:
learning_rate = 0.001

scores = {}

for size in [10, 100, 1000]:
    print(size)

    model = make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history

    print()
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % size))

plt.xticks(np.arange(10))
plt.yticks([0.78, 0.80, 0.82, 0.84, 0.86])
plt.legend()

REGULARIZATION AND **DROPOUT**

In [ ]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.5, input_size=150):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(input_size, input_size, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)

    outputs = keras.layers.Dense(1, activation='sigmoid')(drop)

    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.BinaryCrossentropy(from_logits=False)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [ ]:
learning_rate = 0.001
size = 100

scores = {}

for droprate in [0.0, 0.2]:
    print(droprate)

    model = make_model(
        learning_rate=learning_rate,
        size_inner=size,
        droprate=droprate
    )

    history = model.fit(train_ds, epochs=30, validation_data=val_ds)
    scores[droprate] = history.history

    print()
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % droprate))

plt.ylim(0.5, 0.9)
plt.legend()

In [ ]:
hist = scores[0.0]
plt.plot(hist['val_accuracy'], label=0.0)

hist = scores[0.2]
plt.plot(hist['val_accuracy'], label=0.2)

plt.legend()

LETS NOW TRAIN THE MODEL WITH AUGUMENTATIONS

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=10,
    height_shift_range=10,
    shear_range=10,
    zoom_range=0.1,
    cval=0.0,
    horizontal_flip=False,
    vertical_flip=True,
)

In [ ]:
# Data augmentation for training
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.1,      
    zoom_range=0.1,       
    vertical_flip=True,   
)

train_ds = train_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Validation data (no augmentation)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/val',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    class_mode='binary'
)

In [ ]:
learning_rate = 0.001
size = 100
droprate = 0.2

model = make_model(
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)

history = model.fit(train_ds, epochs=50, validation_data=val_ds)


In [ ]:

hist = history.history
plt.plot(hist['val_accuracy'], label='val')
plt.plot(hist['accuracy'], label='train')

plt.legend()

LETS NOW TRAIN A LARGER MODEL

In [ ]:
def make_model(input_size = input_size,learning_rate=0.01, size_inner=100, droprate=0.5):
    base_model = Xception(
        weights='imagenet',
        include_top=False,
        input_shape=(150, 150, 3)
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)

    outputs = keras.layers.Dense(1, activation='sigmoid')(drop)

    model = keras.Model(inputs, outputs)

    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = keras.losses.BinaryCrossentropy(from_logits=False)

    model.compile(
        optimizer=optimizer,
        loss=loss,
        metrics=['accuracy']
    )

    return model

In [ ]:
input_size = 299

In [ ]:
# Data augmentation for training
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=0.1,      
    zoom_range=0.1,       
    vertical_flip=True,   
)

train_ds = train_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/train',
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

# Validation data (no augmentation)
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/val',
    target_size=(150, 150),
    batch_size=32,
    shuffle=False,
    class_mode='binary'
)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.0005
size = 100
droprate = 0.2

model = make_model(
    input_size = input_size,
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate
)
history = model.fit(train_ds, epochs=40, validation_data=val_ds,
                   callbacks=[checkpoint])

LOADING THE MODEL

In [ ]:
import tensorflow as tf
from tensorflow import keras
 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input

In [ ]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    '/kaggle/input/chest-exray-pneumonia-detection/chest_xray/test',
    target_size=(150,150),
    batch_size=32,
    shuffle=False,
    class_mode='binary'
)


In [ ]:
import os
os.listdir('/kaggle/working')


In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model(
    '/kaggle/working/xception_v4_1_03_0.812.h5'
)



In [ ]:
model.evaluate(test_ds)

In [ ]:
import numpy as np

# Get predictions
preds = model.predict(test_ds)

# Convert probabilities to class labels (0 = Normal, 1 = Pneumonia)
pred_labels = np.where(preds > 0.5, 1, 0)

# Show first 10 predictions
for i in range(10):
    print(f"Predicted: {pred_labels[i][0]}, True: {test_ds.classes[i]}")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# true labels
y_true = test_ds.classes
# predicted labels
y_pred = np.where(model.predict(test_ds) > 0.5, 1, 0).flatten()

# confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", cm)

# classification report
print("\nClassification Report:\n", classification_report(y_true, y_pred, target_names=list(test_ds.class_indices.keys())))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images
images, labels = next(test_ds)

# Convert Xception-preprocessed images back to displayable format
images_disp = (images + 1) * 127.5  # scale [-1,1] → [0,255]

# Show first 5 images
plt.figure(figsize=(15,5))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(images_disp[i].astype('uint8'))
    plt.title('PNEUMONIA' if labels[i]==1 else 'NORMAL')
    plt.axis('off')
plt.show()


load  a single image and make prediction

In [ ]:

# Step 1: Import libraries
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

# Step 2: Load your trained model
model = keras.models.load_model('/kaggle/working/xception_v4_1_03_0.812.h5')

# Step 3: Load and preprocess the new image
img_path = '/kaggle/input/patient-xray/journal.pone.0256630.g001.PNG'

img = load_img(img_path, target_size=(150,150)) 
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_input = preprocess_input(img_array)

# Step 4: Make prediction
pred = model.predict(img_input)
label = 'PNEUMONIA' if pred[0][0] > 0.5 else 'NORMAL'
print(f"Prediction: {label}")

# Step 5: Display the image
plt.imshow(img)
plt.title(f'Predicted: {label}')
plt.axis('off')
plt.show()



In [ ]:
from tensorflow import keras

# Save your trained model
model.save('/kaggle/working/xception_pneumonia_model.keras')


In [ ]:
import os

print(os.listdir('/kaggle/working/'))


In [ ]:
import shutil
os.makedirs('/kaggle/output', exist_ok=True)
shutil.move('/kaggle/working/xception_pneumonia_model.keras', '/kaggle/output/xception_pneumonia_model.keras')

# Check output folder
print(os.listdir('/kaggle/output/'))


In [ ]:
import shutil
shutil.copy('/kaggle/output/xception_pneumonia_model.keras', '/kaggle/working/xception_pneumonia_model.keras')

In [ ]:
from IPython.display import FileLink

# This generates a link for the file you moved to /kaggle/output/
FileLink(r'xception_pneumonia_model.keras')

In [ ]:
model_path = '/kaggle/working/xception_pneumonia_model.keras'
size_mb = os.path.getsize(model_path) / (1024 * 1024)

print(f"Model size: {size_mb:.2f} MB")


In [ ]:
model.summary()
